In [1]:
import torch
from QuGAT import *

In [2]:
## 输入数据 ##
# 单个特征编码的qubit数目
Nqubits=3

# 顶点的特征编码
num_of_vertex = 3 # 顶点数
x1=torch.Tensor([1.,0.,2.,0.,1.,0.,2.,0.]) # 2^(Nqubits) = 8
x2=torch.Tensor([0.,1.,1.,0.,0.,1.,1.,0.])
x3=torch.Tensor([0.,2.,1.,1.,0.,2.,1.,1.])
x = [x1, x2, x3]

# 邻接矩阵
A =([[1,1,0],
     [1,1,1],
     [0,1,1]])

In [3]:
## 计算过程 ##
## 计算x_n -> qx_n, 存在list中 ##

qx=[]
for n in range(num_of_vertex):
    qx.append(vector2rho(x[n]))
# return qx

# print(len(qx))
# print(qx[0].size())

In [4]:
# 构建 QC1
QC1 = Cir_Init_Feature(n_qubits=Nqubits*2) # n_qubits暂定,theta_size暂定6

sigma_z = z_gate()
I = I_gate()
ancillia = torch.tensor([[1,0],[0,0]])

## 计算alpha_ij，融合qx_i, qx_j,存入alpha_list中
alpha_list=[]
for i in range(num_of_vertex):
    alpha_i = []
    for j in A[i]:
        if j: # 两个顶点是连接的
            qx_ij = torch.kron(qx[i],qx[j]) # 线路的输入
            qx_ij_out = QC1.forward(multi_kron([ancillia,qx_ij])) # 通过QC1量子线路
            
            O_M = multi_kron([sigma_z,I,I,I,I,I,I]) # 测量第一个Qubit的\sigma_z力学量算符
            
            alpha_ij = measure(qx_ij_out, O_M) # 进行z方向上的测量
            alpha_i.append(alpha_ij)
        else: # 两个顶点是*不*连接的
            alpha_i.append(torch.tensor(0))
    alpha_list.append(alpha_i)

print(alpha_list)

[[tensor(0.4000+0.j), tensor(0.4000+0.j), tensor(0)], [tensor(1.0000+0.j), tensor(1.0000+0.j), tensor(1.0000+0.j)], [tensor(0), tensor(0.7500+0.j), tensor(0.7500+0.j)]]


In [5]:
## 将alpha_ij放入量子线路
# 这里的alpha_ij是上一个线路的测量值，所以其值域在[0,1]之间。因此需要做一个线性变换，使其能够符合非线性变换的定义域
def Nonlinear(theta):
    # todo 对theta进行非线性变换
    return theta


In [6]:
# 构建量子线路QC2（XYX构型）
# 对每个节点i强化后的特征值作为输入，放入attention score参数量子线路QC2中,最终的输出qax

qax=[]
for i in range(num_of_vertex):
    qaxi=[]
    for theta in alpha_list[i]:
        if theta != torch.tensor(0):
            theta = Nonlinear(theta)
            QC2 = Cir_XYX(theta, n_qubits=Nqubits)
            u_out = QC2.forward(qx[i])
            qaxi.append(u_out)
    qax.append(qaxi)

# return qax
# print(len(qax))
# print(qax[2].__len__())

In [10]:
# 构建量子线路QC3，对两个输入相同维度的量子态进行类Sum操作
QC3 = Cir_Sum(n_qubits=Nqubits*2)

qax_out = []
for i in range(num_of_vertex):
    q_out = qax[i][0]
    for idx in range(len(qax[i])-1): # 对于i所有的邻域节点
        q_in = torch.kron(q_out, qax[i][idx+1]) # 2N个qubit资源
   
        # 通过有CNOT门的量子线路
        q_out = QC3.forward(q_in)

        # pratial_trace 回到同样的维度
        q_out = ptrace(q_out, Nqubits, Nqubits)

    qax_out.append(q_out)


In [7]:
from functools import reduce
# 构建量子线路QC3，对两个输入相同维度的量子态进行类Sum操作
QC3 = Cir_Sum(n_qubits=Nqubits*2)

# 两个相同维度的态相互作用
def sum_two_state(qin1,qin2):
    "对两个相同维度的态，做Sum的量子线路融合"
    qin = torch.kron(qin1,qin2)
    return ptrace(QC3.forward(qin), Nqubits, Nqubits)

q_out = []
for i in range(num_of_vertex):
    qi_out = reduce(sum_two_state, qax[i])
    q_out.append(qi_out)

print(q_out)

[tensor([[ 0.1208-2.3283e-10j,  0.0073-2.5360e-02j, -0.1159+4.4597e-02j,
          0.0023+2.7048e-02j, -0.1540+9.5482e-02j,  0.0107+3.8153e-02j,
          0.0481+5.1788e-02j,  0.0138-6.9419e-03j],
        [ 0.0073+2.5360e-02j,  0.0308-8.7311e-11j, -0.0164-2.1622e-02j,
         -0.0295+1.1361e-02j, -0.0294-2.6535e-02j, -0.0392+2.4324e-02j,
         -0.0080+1.3243e-02j,  0.0122+1.3193e-02j],
        [-0.1159-4.4597e-02j, -0.0164+2.1622e-02j,  0.1776+4.6566e-09j,
          0.0108-3.7285e-02j,  0.2368-4.7650e-02j,  0.0044-5.2635e-02j,
          0.0187-6.7702e-02j, -0.0131-8.0444e-03j],
        [ 0.0023-2.7048e-02j, -0.0295-1.1361e-02j,  0.0108+3.7285e-02j,
          0.0452+2.0373e-10j,  0.0244+4.6837e-02j,  0.0603-1.2139e-02j,
          0.0154-1.9361e-04j,  0.0048-1.7247e-02j],
        [-0.1540-9.5482e-02j, -0.0294+2.6535e-02j,  0.2368+4.7650e-02j,
          0.0244-4.6837e-02j,  0.3967+1.1452e-08j,  0.0241-8.3312e-02j,
          0.0468-1.0329e-01j, -0.0188-1.6122e-02j],
        [ 0.0107-3.

In [12]:
print("量子态输出：",qax_out.__len__())
# print(qax_out)
# print(qax_out[0].size())

量子态输出： 3


In [14]:
sum(qax_out[0] - q_out[0])

tensor([-0.7185-0.0162j, -0.0461+0.0799j,  0.6201-0.1900j,  0.7872-0.3516j,
         0.2272+0.1758j,  0.4164-0.0359j,  0.5210-0.1336j,  0.9367-0.1919j],
       grad_fn=<AddBackward0>)